# Overview

Download radiosonde data and plot vertical profiles of potential temperature.

In [17]:
import act
from tempfile import TemporaryDirectory
import glob
import numpy as np
import pandas as pd
import datetime as dt
import altair as alt

import metpy
from metpy.units import units
import pint_xarray

from sublimpy import utils
import pytz

In [18]:
username = os.getenv("ARM_USERNAME")
token = os.getenv("ARM_TOKEN")
radiosonde ='gucsondewnpnM1.b1'
DATE_FORMAT_STR = '%Y-%m-%d'
start_date = dt.datetime.strptime("2023-06-09", DATE_FORMAT_STR)
end_date = dt.datetime.strptime("2023-06-10", DATE_FORMAT_STR)

In [19]:

# end_date = dt.datetime.today()
pressure_value = 600
pressure_variation = 10
date_list = [start_date + dt.timedelta(days=i) for i in range(0,(end_date - start_date).days)]
ds_list = []
df = pd.DataFrame()
with TemporaryDirectory() as temp_dir:
    for date in date_list:
        date_str = dt.datetime.strftime(date, DATE_FORMAT_STR)
        files = act.discovery.download_arm_data(
            username,    token,    radiosonde,    date_str, date_str, output = temp_dir)
        sonde_ds = act.io.read_arm_netcdf(files)
        ds_list.append(sonde_ds)


[DOWNLOADING] gucsondewnpnM1.b1.20230609.113000.cdf
[DOWNLOADING] gucsondewnpnM1.b1.20230609.234000.cdf

If you use these data to prepare a publication, please cite:

Keeler, E., Burk, K., & Kyrouac, J. Balloon-Borne Sounding System (SONDEWNPN).
Atmospheric Radiation Measurement (ARM) User Facility.
https://doi.org/10.5439/1595321



In [20]:
df_list = [ds.to_dataframe() for ds in ds_list]
df = pd.concat(df_list)
# add potential temp calculations
df['dp_potential'] = metpy.calc.potential_temperature(
    df['pres'].to_xarray()*units('hPa'),
    df['dp'].to_xarray()*units('degC')
).pint.to('degC')

df['tdry_potential'] = metpy.calc.potential_temperature(
    df['pres'].to_xarray()*units('hPa'),
    df['tdry'].to_xarray()*units('degC')
).pint.to('degC')

In [21]:
df = df.reset_index()
df['hour'] = df.time.dt.hour

In [22]:
elevation_gothic_m = 2884.322
src = df.copy()
src = utils.modify_df_timezone(src, pytz.UTC, 'US/Mountain')
src['alt_relative'] = src['alt'] - elevation_gothic_m
src = src[src['alt_relative'] <= 2000]
src['hour'] = src.time.dt.hour
src['is_high'] = src['alt_relative'] > 600
(alt.Chart(
    src.rename(columns={'tdry_potential':'dry bulb temp'})
).transform_fold(['dry bulb temp']).mark_line(color='orange').encode(
    alt.X("value:Q").sort('-y').title("Potential temperature (˚C)"),
    alt.Y("alt_relative:Q").scale(zero=False),
    alt.StrokeDash('is_high:Q', legend=None).scale(range = [[], [2,4]]),
    alt.Color("hour:N")
)).resolve_scale(x='independent').properties(width=200, height = 200).configure_legend(
    orient='top',
    direction='horizontal',
    titleAnchor='middle'
).resolve_scale(y='shared')

alt.Chart(...)

In [23]:
elevation_gothic_m = 2884.322
src = df.copy()
src = utils.modify_df_timezone(src, pytz.UTC, 'US/Mountain')
src['alt_relative'] = src['alt'] - elevation_gothic_m
src = src[src['alt_relative'] <= 2000]
src['hour'] = src.time.dt.hour

In [8]:
alt.Chart(
    src[['wspd', 'alt_relative', 'hour']]
).mark_line().encode(
    alt.X("wspd:Q").sort('-y'),
    alt.Y("alt_relative:Q"),
    alt.Color("hour:N")
).properties(title=str(src.time.dt.date.iloc[0]))

alt.Chart(...)